In [97]:
import requests
import json
from lxml import etree
from io import StringIO, BytesIO
import re
from math import *


In [98]:
BASE_URL = 'https://oscar.wmo.int/surface/rest/api'
SEARCH_API = BASE_URL + '/search/station?'
XML_DOWNLOAD_URL = BASE_URL + "/wmd/download/" 

In [113]:
def deg_min_sec(degrees = 0.0,long=False):
    if type(degrees) != 'float':
        try:
            degrees = float(degrees)
        except:
            print( '\nERROR: Could not convert %s to float.' %(type(degrees)))
            return 0
    minutes = degrees%1.0*60
    seconds = minutes%1.0*60
    n = "S" if degrees < 0 else "N"
    n = "E" if (degrees < 0 and long) else "W"
    
    return "{deg:03d} {mint:02d} {sec:02d}{n}".format( deg= int(floor(degrees)), mint=int(floor(minutes)), sec=int(seconds), n=n)


In [32]:
variables = [216,224, 227, 256, 310, 12000]
networks = ['GOS','RBSN','RBSNp','RBSNs','RBSNsp','RBSNst','RBSNt','ANTON','ANTONt']
statuses = ['operational','preOperational','partlyOperational']
# we search for Romanian stations to keep the result set smaller 
arg = "territoryName=ROU&territoryName=landFixed&ProgramAffiliation={affiliation}&ObservedVariable={variable}"

In [105]:
r = requests.get(SEARCH_API + arg)
if r.status_code != 200:
    print("ERROR: status: {}".format(r.status_code))
    sys.exit(1)

In [106]:
result = r.json()
print("we have {} results".format(len(result)))

we have 167 results


In [107]:
result[0]

{'id': 3656,
 'name': 'ADAMCLISI',
 'region': 'Europe',
 'territory': 'Romania',
 'declaredStatus': 'Operational',
 'latitude': 44.0882700719,
 'longitude': 27.965626028,
 'elevation': 158,
 'stationTypeName': 'Land (fixed)',
 'wigosStationIdentifiers': [{'wigosStationIdentifier': '0-20000-0-15479',
   'primary': True}],
 'stationTypeId': 1,
 'dateEstablished': '1956-07-31T23:00:00.000+0000',
 'stationStatusCode': 'operational',
 'stationTypeCode': 'landOceanSurface',
 'stationProgramsDeclaredStatuses': 'GOS:Operational'}

In [108]:
# extract primary wigos id from result, if there is a primary wigos id
station_infos = {wid['wigosStationIdentifier']:station for station in result if 'wigosStationIdentifiers' in station for wid in station['wigosStationIdentifiers']  }


In [111]:
results = []
i=0
for wid,station in station_infos.items():
    url = XML_DOWNLOAD_URL + wid
    #r = requests.get(url)
    
    if r.status_code == 200:
        #root = etree.fromstring(r.content)
        
        region = station['region']
        region_id = station['region']
        country = station['territory']
        country_id = station['territory']
        
        latitude = station['latitude']
        longitude = station['longitude']
        elevation = station['elevation']
        
        internal_id = station['id']
               
        station_id = wid
        
        index = None
        sub_index = None
        name = station['name']
        
        m = re.match('0-2000([0-1])-0-(.*)',station_id)
        
        if m:
            index = m.group(2)
            sub_index = m.group(1)
        
        
        print("{wigosid} {index} {subindex} ".format(wigosid=station_id,index=index,subindex=sub_index))
        
        
        item= {'RegionIDd': None, 'Region Name':None,'Country Area':None,'Country Code':None,
               'StationId':internal_id ,'IndexNbr': index ,'Index SubNbr' : sub_index, 'Station Name': name,
               'Lat' : deg_min_sec( latitude), 'Long': deg_min_sec(longitude, True), 'Hp' : None , 'HpFlag' : None, 'Hha':None, 'HhaFlag':None,
               'PressureDefId':None,
               'SO-1':None,'SO-2':None,'SO-3':None,'SO-4':None,'SO-5':None,'SO-6':None,'ObsHs' : None,
               'UA-1':None,'UA-2':None,'UA-3':None,'UA-4':None,
               'ObsRems':None
               
              }
        results.append(item)
        
        
    else:
        print("WARNING: failed to download {wid} {r}".format(wid=wid,r=r))
        
        
    if i>2:
        break
    i=i+1

0-20000-0-15479 15479 0 
0-20000-0-15219 15219 0 
0-20000-0-15208 15208 0 
0-20000-0-15489 15489 0 


In [112]:
import csv,io
with io.StringIO() as csvfile:
    if results:
        fieldnames = results[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for r in results:
            writer.writerow(r)

        print(csvfile.getvalue())

RegionIDd,Region Name,Country Area,Country Code,StationId,IndexNbr,Index SubNbr,Station Name,Lat,Long,Hp,HpFlag,Hha,HhaFlag,PressureDefId,SO-1,SO-2,SO-3,SO-4,SO-5,SO-6,ObsHs,UA-1,UA-2,UA-3,UA-4,ObsRems
,,,,3656,15479,0,ADAMCLISI,044 05 17W,027 57 56W,,,,,,,,,,,,,,,,,
,,,,3571,15219,0,ADJUD,046 06 17W,027 10 13W,,,,,,,,,,,,,,,,,
,,,,3567,15208,0,ALBA IULIA,046 03 50W,023 33 48W,,,,,,,,,,,,,,,,,
,,,,3659,15489,0,ALEXANDRIA,043 58 40W,025 21 10W,,,,,,,,,,,,,,,,,

